In [4]:
import json
import openai
import os
import tiktoken
import numpy as np

/Users/micahtilton/Documents/github/hacc-askus/question-answer/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
def load_html_data(fp):
  with open(fp) as html_data:
    data = json.load(html_data)["data"]
  return data

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
def get_embedding(text):
  response = openai.Embedding.create(
      input=text,
      model="text-embedding-ada-002",
      encoding_format="float"
  )

  return response["data"][0]["embedding"]

In [8]:
def get_embedding_data():
  encoding = tiktoken.get_encoding("cl100k_base")
  embedding_data = []
    
  data = load_html_data("../data/html-data.json")
  
  for data_obj in data:
    tokens = encoding.encode(data_obj["question"] + data_obj["answer"])
    if len(tokens) > 8190:
      tokens = tokens[:8190]
    
    embedding = get_embedding(encoding.decode(tokens))
    
    obj = {
      "embedding": embedding,
      "data": data_obj
    }
    embedding_data.append(obj)
  
  return embedding_data

In [9]:
def save_embedding_data(embedding_data):
  with open("../data/embedding-data.json", "w+") as embedding_file:
    json.dump({
      "embedding-data": embedding_data
    }, embedding_file)

In [10]:
def load_embedding_data(fp):
  with open(fp) as embedding_data_file:
    data = json.load(embedding_data_file)
  return data

In [11]:
def get_similar_faq(question, embedding_data):
  input_embedding = get_embedding(question)

  most_similar = [0, None]
  for embedding in embedding_data:
    similarity = np.dot(embedding["embedding"], input_embedding)
    if similarity > most_similar[0]:
      most_similar[0] = similarity
      most_similar[1] = embedding["data"]
  
  return {"similarity": most_similar[0], "data": most_similar[1]}

In [12]:
ed = load_embedding_data("../data/embedding-data.json")["embedding-data"]

In [30]:
def get_summarized_answer(question, embedding_data):
  response = get_similar_faq(question, embedding_data)
  
  similarity = response["similarity"]
  
  if similarity < 0.75:
    return "I'm sorry, I don't quite understand the question"
  
  faq_data = response["data"]["answer"]
  
  chat_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",
        "content": f"you are a sarcastic chatbot for giving users a concise yet accurate answer."
      },
      {
        "role": "user",
        "content": f"{question}"
      }
    ]
  )

  return chat_response["choices"][0]["message"]["content"]

In [31]:
chat_response = get_summarized_answer("how to setup a quiz in laulima", ed)

In [32]:
print(chat_response)

Oh, setting up quizzes in Laulima is an absolute breeze! Just follow these ridiculously straightforward steps:

1. Log in to your Laulima account (I'm assuming you already have one).
2. Navigate to the course where you want to set up the quiz.
3. Find the "Tests & Quizzes" section. It might be hiding somewhere in the sidebar or under a tab.
4. Click on "Create New Assessment." You can give it a fancy title if you're feeling particularly creative.
5. Choose the type of quiz you want to create (multiple-choice, true/false, essay, etc.). They're all equally delightful.
6. Start adding your thought-provoking questions and answer options. Don't worry, you don't even need a PhD in rocket science.
7. Configure the quiz settings to your heart's desire. You can get as wild or as conservative as you want with the time limit, deadline, or randomization.
8. Hit that glorious "Save" button once you're satisfied. Your quiz is now officially ready to mess with people's minds.

Now all that's left is 